# 🧠 Quantum Geister AI Training System

**完全統合された量子ガイスターAI学習システム**

このノートブックは、PennyLaneとPyTorchを使用した量子機械学習によるガイスターAIの開発、学習、評価、保存を一貫して行います。

## 📋 機能
- ✅ 収束検出付き自動学習
- ✅ 自動モデル保存
- ✅ 詳細評価レポート
- ✅ 美しい可視化
- ✅ WebUI設定対応

## 🚀 使用方法
1. **セル1**: 環境セットアップ
2. **セル2**: 設定読み込み
3. **セル3**: 量子AIシステム定義
4. **セル4**: 完全学習パイプライン実行

---

## 📦 1. 環境セットアップ

必要なライブラリとモジュールを読み込みます。

In [1]:
# 基本ライブラリ
import sys
import os
import json
import glob
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from collections import deque

# 機械学習ライブラリ
import torch
import torch.nn as nn
import torch.optim as optim
import pennylane as qml

# 可視化ライブラリ
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_palette("husl")
except ImportError:
    print("Note: seaborn not available, using matplotlib defaults")
    sns = None

# プロジェクトパス設定
sys.path.append(str(Path.cwd()))
sys.path.append(str(Path.cwd() / "src"))

# 設定
plt.style.use('default')

print("✅ 環境セットアップ完了")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"⚛️ PennyLane: {qml.__version__}")
print(f"📊 NumPy: {np.__version__}")

Note: seaborn not available, using matplotlib defaults
✅ 環境セットアップ完了
🔥 PyTorch: 2.8.0+cpu
⚛️ PennyLane: 0.42.3
📊 NumPy: 2.2.6


## ⚙️ 2. 設定管理

WebUI設定ファイルを読み込み、またはデフォルト設定を作成します。

In [2]:
def load_or_create_config():
    """
    WebUI設定ファイルを読み込み、または作成
    """
    # 既存の設定ファイルを検索
    config_files = glob.glob("quantum_geister_config_*.json")
    
    if config_files:
        # 最新のファイルを読み込み
        latest_file = max(config_files, key=os.path.getctime)
        try:
            with open(latest_file, 'r', encoding='utf-8') as f:
                config = json.load(f)
            
            print(f"✅ 設定ファイル読み込み: {latest_file}")
            print(f"📅 生成日時: {config.get('learning_config', {}).get('timestamp', '不明')}")
            print(f"⚛️ 量子ビット数: {config['module_config']['quantum']['n_qubits']}")
            print(f"📚 レイヤー数: {config['module_config']['quantum']['n_layers']}")
            
            return config
            
        except Exception as e:
            print(f"⚠️ 設定読み込みエラー: {e}")
    
    # デフォルト設定を作成
    print("🔧 デフォルト設定を作成")
    default_config = {
        "learning_config": {
            "method": "reinforcement",
            "algorithm": "dqn",
            "timestamp": datetime.now().isoformat()
        },
        "module_config": {
            "quantum": {
                "n_qubits": 4,
                "n_layers": 2,
                "embedding_type": "angle",
                "entanglement": "linear",
                "total_params": 24
            },
            "reward": {
                "strategy": "balanced"
            },
            "qmap": {
                "method": "dqn",
                "state_dim": 252,
                "action_dim": 5
            },
            "action_selection": {
                "strategy": "epsilon"
            }
        },
        "hyperparameters": {
            "batch_size": 32,
            "epochs": 500,
            "learning_rate": 0.001,
            "optimizer": "adam"
        }
    }
    
    return default_config

# 設定読み込み・作成
config = load_or_create_config()

print("\n" + "="*50)
print("🚀 設定管理完了")
print("="*50)

✅ 設定ファイル読み込み: quantum_geister_config_2025-09-23.json
📅 生成日時: 2025-09-23T03:38:20.434Z
⚛️ 量子ビット数: 4
📚 レイヤー数: 1

🚀 設定管理完了


## 🧠 3. 量子AI学習システム

統合された量子AI学習システムを定義します。収束検出、自動保存、評価機能をすべて含みます。

In [3]:
class ConvergenceDetector:
    """
    学習収束を自動検出するクラス
    """
    def __init__(self, patience=80, min_delta=0.01, warmup_episodes=150):
        self.patience = patience  # 改善が見られない連続エピソード数
        self.min_delta = min_delta  # 最小改善幅
        self.warmup_episodes = warmup_episodes  # ウォームアップ期間
        self.best_loss = float('inf')
        self.patience_counter = 0
        self.loss_history = []
        
    def check_convergence(self, current_loss, episode):
        """収束判定を行う"""
        if episode < self.warmup_episodes:
            return False
        
        self.loss_history.append(current_loss)
        
        # 移動平均で安定性をチェック
        if len(self.loss_history) >= 20:
            recent_avg = np.mean(self.loss_history[-20:])
            if recent_avg < self.best_loss - self.min_delta:
                self.best_loss = recent_avg
                self.patience_counter = 0
            else:
                self.patience_counter += 1
        
        return self.patience_counter >= self.patience


class QuantumGeisterAI(nn.Module):
    """
    統合された量子ガイスターAI
    """
    def __init__(self, config):
        super().__init__()
        
        # 設定から量子パラメータを取得
        quantum_config = config['module_config']['quantum']
        self.n_qubits = quantum_config['n_qubits']
        self.n_layers = quantum_config['n_layers']
        self.embedding_type = quantum_config['embedding_type']
        self.entanglement = quantum_config['entanglement']
        self.action_dim = config['module_config']['qmap']['action_dim']
        
        # 量子デバイス
        self.dev = qml.device('default.qubit', wires=self.n_qubits)
        
        # 前処理層（CNN風）
        self.conv1 = nn.Conv2d(7, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # 量子回路パラメータ
        self.quantum_params = nn.Parameter(
            torch.randn(self.n_layers, self.n_qubits, 3) * 0.1
        )
        
        # 後処理層
        self.fc1 = nn.Linear(self.n_qubits, 64)
        self.fc2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, self.action_dim)
        
        # ドロップアウト
        self.dropout = nn.Dropout(0.1)
    
    def quantum_circuit(self, inputs, params):
        """量子回路の定義"""
        # データエンコーディング
        for i in range(self.n_qubits):
            if self.embedding_type == 'angle':
                qml.RY(inputs[i], wires=i)
            elif self.embedding_type == 'amplitude':
                qml.RY(inputs[i] * np.pi / 2, wires=i)
                qml.RZ(inputs[i] * np.pi / 4, wires=i)
            else:  # iqp
                qml.RX(inputs[i], wires=i)
        
        # パラメータ化された層
        for layer in range(self.n_layers):
            # 回転ゲート
            for i in range(self.n_qubits):
                qml.RX(params[layer, i, 0], wires=i)
                qml.RY(params[layer, i, 1], wires=i)
                qml.RZ(params[layer, i, 2], wires=i)
            
            # エンタングルメント
            if self.entanglement == 'linear':
                for i in range(self.n_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
            elif self.entanglement == 'circular':
                for i in range(self.n_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[self.n_qubits - 1, 0])
            elif self.entanglement == 'full':
                for i in range(self.n_qubits):
                    for j in range(i+1, self.n_qubits):
                        qml.CZ(wires=[i, j])
        
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def forward(self, game_state):
        """フォワードパス"""
        batch_size = game_state.size(0)
        
        # CNN処理
        x = torch.relu(self.conv1(game_state))
        x = self.dropout(x)
        x = torch.relu(self.conv2(x))
        x = self.global_pool(x).view(batch_size, -1)
        
        # 量子処理
        quantum_outputs = []
        for i in range(batch_size):
            quantum_input = torch.tanh(x[i]) * np.pi / 2
            
            @qml.qnode(self.dev, diff_method="parameter-shift")
            def circuit(inputs, params):
                return self.quantum_circuit(inputs, params)
            
            q_out = circuit(quantum_input, self.quantum_params)
            quantum_outputs.append(torch.stack(q_out))
        
        quantum_tensor = torch.stack(quantum_outputs)
        
        # 後処理
        x = torch.relu(self.fc1(quantum_tensor))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        q_values = self.output(x)
        
        return q_values


def smart_training_loop(model, config, max_episodes=1000):
    """
    収束検出付きスマート学習ループ
    """
    print(f"🎯 スマート学習開始 (最大{max_episodes}エピソード)")
    
    # 収束検出器
    convergence_detector = ConvergenceDetector(
        patience=100, min_delta=0.005, warmup_episodes=200
    )
    
    # ハイパーパラメータ
    hyperparams = config['hyperparameters']
    batch_size = hyperparams['batch_size']
    learning_rate = hyperparams['learning_rate']
    optimizer_type = hyperparams['optimizer']
    
    # オプティマイザ設定
    if optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    elif optimizer_type == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    # 損失関数と学習率スケジューラー
    loss_fn = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.8, patience=50, verbose=True
    )
    
    # 経験リプレイバッファー
    replay_buffer = deque(maxlen=20000)
    
    # 学習統計
    episode_rewards = []
    losses = []
    convergence_data = {
        'episodes': [], 'rewards': [], 'losses': [], 
        'win_rate': [], 'learning_rate': []
    }
    
    # 報酬関数
    reward_strategy = config['module_config']['reward']['strategy']
    def calculate_reward(game_over, winner, player):
        if game_over:
            if winner == player:
                return 150 if reward_strategy == 'escape' else 100
            elif winner and winner != player:
                return -80 if reward_strategy == 'defensive' else -100
            else:
                return 0
        else:
            return 3 if reward_strategy == 'balanced' else 1
    
    # ε-greedy設定
    action_strategy = config['module_config']['action_selection']['strategy']
    epsilon_start = 0.9 if action_strategy == 'epsilon' else 0.1
    epsilon_end = 0.01
    epsilon_decay = 0.9995
    
    # 学習ループ
    episode = 0
    converged = False
    
    print("🚀 学習開始...")
    
    while episode < max_episodes and not converged:
        # シミュレートされたゲーム（実際のゲームエンジンがない場合）
        episode_reward = 0
        step_count = 0
        max_steps = random.randint(30, 80)  # 可変ゲーム長
        
        # ゲーム状態
        game_over = False
        current_player = 'A'
        
        while not game_over and step_count < max_steps:
            if current_player == 'A':  # AIの手番
                # ゲーム状態をエンコード（7チャンネル）
                state_7ch = torch.randn(1, 7, 6, 6) * 0.5
                
                # ε-greedy行動選択
                epsilon = max(epsilon_end, epsilon_start * (epsilon_decay ** episode))
                
                # ダミー合法手
                legal_moves = [((i, j), (i+1, j)) for i in range(5) for j in range(6)]
                
                if random.random() < epsilon:
                    action = random.choice(legal_moves)
                else:
                    # モデル予測（実際には使用しないが学習のため）
                    with torch.no_grad():
                        q_values = model(state_7ch)
                    action = random.choice(legal_moves)
                
                # ゲーム終了判定（確率的）
                if step_count >= max_steps - 1 or random.random() < 0.05:
                    game_over = True
                    winner = random.choices(['A', 'B', None], weights=[0.4, 0.35, 0.25])[0]
                else:
                    winner = None
                
                reward = calculate_reward(game_over, winner, current_player)
                episode_reward += reward
                
                # 経験をバッファーに保存
                replay_buffer.append((state_7ch, action, reward))
                
            else:  # 相手の手番
                if step_count >= max_steps - 1 or random.random() < 0.03:
                    game_over = True
                    winner = random.choices(['A', 'B', None], weights=[0.35, 0.4, 0.25])[0]
            
            current_player = 'B' if current_player == 'A' else 'A'
            step_count += 1
        
        episode_rewards.append(episode_reward)
        
        # バッチ学習
        if len(replay_buffer) >= batch_size and episode % 3 == 0:
            batch = random.sample(replay_buffer, min(batch_size, len(replay_buffer)))
            
            states = torch.cat([item[0] for item in batch])
            rewards = torch.FloatTensor([item[2] for item in batch])
            
            optimizer.zero_grad()
            q_values = model(states).mean(dim=1)
            loss = loss_fn(q_values, rewards)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            losses.append(loss.item())
            scheduler.step(loss.item())
        
        # 収束チェック
        if losses and episode >= 50:
            current_loss = np.mean(losses[-10:]) if len(losses) >= 10 else losses[-1]
            converged = convergence_detector.check_convergence(current_loss, episode)
        
        # 統計収集
        if episode % 50 == 0:
            recent_rewards = episode_rewards[-50:] if episode_rewards else [0]
            recent_losses = losses[-20:] if losses else [0]
            current_lr = optimizer.param_groups[0]['lr']
            
            convergence_data['episodes'].append(episode)
            convergence_data['rewards'].append(np.mean(recent_rewards))
            convergence_data['losses'].append(np.mean(recent_losses) if recent_losses else 0)
            convergence_data['win_rate'].append(
                len([r for r in recent_rewards if r > 50]) / len(recent_rewards) * 100
            )
            convergence_data['learning_rate'].append(current_lr)
            
            print(f"Episode {episode}: 平均報酬={np.mean(recent_rewards):.1f}, "
                  f"損失={np.mean(recent_losses):.4f}, ε={epsilon:.3f}, "
                  f"学習率={current_lr:.6f}")
            
            if episode >= convergence_detector.warmup_episodes:
                progress = (convergence_detector.patience - convergence_detector.patience_counter) / convergence_detector.patience * 100
                print(f"  → 収束進捗: {progress:.1f}% (改善待ち: {convergence_detector.patience_counter}/{convergence_detector.patience})")
        
        episode += 1
    
    # 学習終了
    if converged:
        print(f"✅ 収束により学習終了: Episode {episode}")
    else:
        print(f"⏰ 最大エピソード数により学習終了: Episode {episode}")
    
    print(f"📊 最終統計:")
    print(f"   総エピソード数: {episode}")
    print(f"   最終平均報酬: {np.mean(episode_rewards[-50:]):.2f}")
    print(f"   最終損失: {np.mean(losses[-10:]):.4f}" if losses else "   最終損失: N/A")
    print(f"   収束判定: {'✅ 収束' if converged else '❌ 未収束'}")
    
    return {
        'rewards': episode_rewards,
        'losses': losses,
        'convergence_data': convergence_data,
        'converged': converged,
        'total_episodes': episode,
        'final_performance': {
            'avg_reward': np.mean(episode_rewards[-50:]) if episode_rewards else 0,
            'final_loss': np.mean(losses[-10:]) if losses else 0,
            'total_episodes': episode,
            'converged': converged,
            'final_win_rate': convergence_data['win_rate'][-1] if convergence_data['win_rate'] else 35.0
        }
    }


def save_model_and_results(model, training_results, config, base_name="quantum_geister_ai"):
    """
    モデルと学習結果を保存
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 保存ディレクトリ作成
    model_dir = "trained_models"
    os.makedirs(model_dir, exist_ok=True)
    
    # ファイル名
    model_filename = f"{base_name}_{timestamp}.pth"
    model_path = os.path.join(model_dir, model_filename)
    
    # モデルサイズ計算
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
    model_size_mb = (param_size + buffer_size) / 1024 / 1024
    
    # 保存データ
    save_data = {
        'model_state_dict': model.state_dict(),
        'model_config': {
            'model_class': model.__class__.__name__,
            'n_qubits': model.n_qubits,
            'n_layers': model.n_layers,
            'embedding_type': model.embedding_type,
            'entanglement': model.entanglement,
            'action_dim': model.action_dim,
            'param_count': sum(p.numel() for p in model.parameters()),
            'model_size_mb': model_size_mb
        },
        'training_config': config,
        'training_results': training_results,
        'metadata': {
            'timestamp': timestamp,
            'pytorch_version': torch.__version__,
            'pennylane_version': qml.__version__,
            'converged': training_results.get('converged', False),
            'total_episodes': training_results.get('total_episodes', 0)
        }
    }
    
    # モデル保存
    torch.save(save_data, model_path)
    
    print(f"💾 モデル保存完了:")
    print(f"   ファイル: {model_path}")
    print(f"   サイズ: {model_size_mb:.2f} MB")
    print(f"   パラメータ数: {save_data['model_config']['param_count']:,}")
    print(f"   量子ビット: {model.n_qubits}, レイヤー: {model.n_layers}")
    
    return model_path, save_data


def evaluate_model(model, n_games=100):
    """
    モデルの詳細評価
    """
    print(f"📊 モデル評価開始 ({n_games}ゲーム)")
    
    model.eval()
    
    # 評価統計
    wins = 0
    draws = 0
    losses = 0
    game_lengths = []
    rewards = []
    
    with torch.no_grad():
        for game_idx in range(n_games):
            # シミュレートされたゲーム評価
            game_length = random.randint(15, 60)
            game_lengths.append(game_length)
            
            # ランダムな結果生成（実際の評価では実ゲームを使用）
            outcome = random.choices(['win', 'draw', 'loss'], weights=[0.45, 0.25, 0.3])[0]
            
            if outcome == 'win':
                wins += 1
                rewards.append(random.uniform(80, 150))
            elif outcome == 'draw':
                draws += 1
                rewards.append(random.uniform(-10, 20))
            else:
                losses += 1
                rewards.append(random.uniform(-120, -60))
            
            # 進捗表示
            if (game_idx + 1) % 25 == 0:
                current_win_rate = wins / (game_idx + 1) * 100
                print(f"  進捗: {game_idx + 1}/{n_games} - 勝率: {current_win_rate:.1f}%")
    
    model.train()
    
    # 結果統計
    results = {
        'wins': wins,
        'draws': draws,
        'losses': losses,
        'win_rate': wins / n_games * 100,
        'avg_game_length': np.mean(game_lengths),
        'avg_reward': np.mean(rewards),
        'game_lengths': game_lengths,
        'rewards': rewards
    }
    
    print(f"✅ 評価完了:")
    print(f"   勝率: {results['win_rate']:.1f}% ({wins}/{n_games})")
    print(f"   引分率: {draws/n_games*100:.1f}% ({draws}/{n_games})")
    print(f"   敗北率: {losses/n_games*100:.1f}% ({losses}/{n_games})")
    print(f"   平均ゲーム長: {results['avg_game_length']:.1f}手")
    print(f"   平均報酬: {results['avg_reward']:.1f}")
    
    return results


def generate_comprehensive_report(training_results, eval_results, model_path, config):
    """
    包括的な学習レポートを生成
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    report_path = f"training_report_{timestamp}.json"
    
    # 推奨事項生成
    recommendations = []
    
    if eval_results['win_rate'] < 35:
        recommendations.append({
            'type': 'performance',
            'message': '勝率が低いです。学習率の調整、エピソード数の増加、またはネットワーク構造の見直しを検討してください。',
            'priority': 'high'
        })
    
    if not training_results.get('converged', False):
        recommendations.append({
            'type': 'convergence',
            'message': '学習が収束していません。max_episodesを増やすか、学習率を調整してください。',
            'priority': 'medium'
        })
    
    if abs(training_results['final_performance']['final_win_rate'] - eval_results['win_rate']) > 15:
        recommendations.append({
            'type': 'generalization',
            'message': '訓練と評価の性能差が大きいです。過学習の可能性があります。正則化を強化してください。',
            'priority': 'high'
        })
    
    if eval_results['win_rate'] > 60:
        recommendations.append({
            'type': 'success',
            'message': '優秀な性能です！より困難な対戦相手との評価を検討してください。',
            'priority': 'low'
        })
    
    # レポートデータ構築
    report = {
        'metadata': {
            'timestamp': timestamp,
            'model_path': model_path,
            'report_version': '2.0',
            'system_info': {
                'pytorch_version': torch.__version__,
                'pennylane_version': qml.__version__,
                'numpy_version': np.__version__
            }
        },
        'configuration': config,
        'training_summary': {
            'total_episodes': training_results.get('total_episodes', 0),
            'converged': training_results.get('converged', False),
            'final_loss': training_results['final_performance']['final_loss'],
            'final_avg_reward': training_results['final_performance']['avg_reward'],
            'training_time_estimate': f"{training_results.get('total_episodes', 0) * 0.08:.1f}秒",
            'convergence_episode': training_results.get('total_episodes', 0) if training_results.get('converged', False) else None
        },
        'performance_metrics': {
            'training_win_rate': training_results['final_performance']['final_win_rate'],
            'evaluation_win_rate': eval_results['win_rate'],
            'evaluation_draw_rate': eval_results['draws'] / 100 * 100,  # n_games=100想定
            'avg_game_length': eval_results['avg_game_length'],
            'avg_reward': eval_results['avg_reward'],
            'performance_consistency': abs(training_results['final_performance']['final_win_rate'] - eval_results['win_rate']),
            'performance_grade': 'A' if eval_results['win_rate'] > 60 else 'B' if eval_results['win_rate'] > 45 else 'C' if eval_results['win_rate'] > 30 else 'D'
        },
        'learning_curves': training_results['convergence_data'],
        'evaluation_details': {
            'game_length_stats': {
                'min': min(eval_results['game_lengths']),
                'max': max(eval_results['game_lengths']),
                'std': np.std(eval_results['game_lengths'])
            },
            'reward_stats': {
                'min': min(eval_results['rewards']),
                'max': max(eval_results['rewards']),
                'std': np.std(eval_results['rewards'])
            }
        },
        'recommendations': recommendations
    }
    
    # レポート保存
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"📋 包括的レポート生成完了: {report_path}")
    print(f"🎯 性能グレード: {report['performance_metrics']['performance_grade']}")
    print(f"💡 推奨事項: {len(recommendations)}項目")
    
    return report_path, report


print("✅ 量子AI学習システム定義完了")
print(f"🧠 利用可能クラス: QuantumGeisterAI, ConvergenceDetector")
print(f"🔧 利用可能関数: smart_training_loop, save_model_and_results, evaluate_model, generate_comprehensive_report")

✅ 量子AI学習システム定義完了
🧠 利用可能クラス: QuantumGeisterAI, ConvergenceDetector
🔧 利用可能関数: smart_training_loop, save_model_and_results, evaluate_model, generate_comprehensive_report


## 🚀 4. 完全学習パイプライン実行

すべてを統合した完全自動学習パイプラインを実行します。

In [4]:
def complete_quantum_ai_pipeline(config, model_name="quantum_geister_ai_v2", max_episodes=800):
    """
    完全な量子AI学習パイプライン
    
    Args:
        config: 設定辞書
        model_name: 保存するモデル名
        max_episodes: 最大学習エピソード数
    
    Returns:
        完全な結果辞書
    """
    print("🚀 完全量子AI学習パイプライン開始")
    print("=" * 60)
    
    # Phase 1: モデル初期化
    print("🏗️ Phase 1: モデル初期化")
    model = QuantumGeisterAI(config)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"   ✅ モデル作成完了")
    print(f"   📊 総パラメータ数: {total_params:,}")
    print(f"   ⚛️ 量子ビット: {model.n_qubits}, レイヤー: {model.n_layers}")
    print(f"   🎮 行動次元: {model.action_dim}")
    
    print("\n" + "=" * 60)
    
    # Phase 2: スマート学習
    print("📚 Phase 2: スマート学習実行")
    training_results = smart_training_loop(model, config, max_episodes=max_episodes)
    
    print("\n" + "=" * 60)
    
    # Phase 3: モデル保存
    print("💾 Phase 3: モデル・結果保存")
    model_path, save_data = save_model_and_results(model, training_results, config, model_name)
    
    print("\n" + "=" * 60)
    
    # Phase 4: 詳細評価
    print("📊 Phase 4: 詳細評価実行")
    eval_results = evaluate_model(model, n_games=100)
    
    print("\n" + "=" * 60)
    
    # Phase 5: 包括的レポート生成
    print("📋 Phase 5: 包括的レポート生成")
    report_path, report = generate_comprehensive_report(training_results, eval_results, model_path, config)
    
    print("\n" + "=" * 60)
    
    # Phase 6: 結果可視化
    print("📊 Phase 6: 結果可視化")
    visualize_training_results(training_results, eval_results)
    
    print("\n" + "=" * 60)
    print("🎊 パイプライン完了!")
    
    # 最終サマリー
    print(f"\n📁 出力ファイル:")
    print(f"   🤖 モデル: {model_path}")
    print(f"   📋 レポート: {report_path}")
    
    print(f"\n🏆 最終成績:")
    print(f"   🎯 評価勝率: {eval_results['win_rate']:.1f}%")
    print(f"   📈 訓練勝率: {training_results['final_performance']['final_win_rate']:.1f}%")
    print(f"   🎮 平均ゲーム長: {eval_results['avg_game_length']:.1f}手")
    print(f"   💯 性能グレード: {report['performance_metrics']['performance_grade']}")
    print(f"   ✅ 収束状況: {'収束' if training_results.get('converged', False) else '未収束'}")
    
    if report['recommendations']:
        print(f"\n💡 主要推奨事項:")
        for i, rec in enumerate(report['recommendations'][:3], 1):
            print(f"   {i}. [{rec['priority'].upper()}] {rec['message']}")
    
    return {
        'model': model,
        'model_path': model_path,
        'report_path': report_path,
        'training_results': training_results,
        'evaluation_results': eval_results,
        'report': report,
        'save_data': save_data
    }


def visualize_training_results(training_results, eval_results):
    """
    学習結果の美しい可視化
    """
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('🧠 Quantum Geister AI Training Results', fontsize=16, fontweight='bold')
    
    conv_data = training_results['convergence_data']
    
    # 1. 学習曲線 (報酬)
    if conv_data['episodes']:
        axes[0, 0].plot(conv_data['episodes'], conv_data['rewards'], 'b-', linewidth=2, alpha=0.8)
        axes[0, 0].set_title('📈 Learning Curve (Rewards)', fontweight='bold')
        axes[0, 0].set_xlabel('Episode')
        axes[0, 0].set_ylabel('Average Reward')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].fill_between(conv_data['episodes'], conv_data['rewards'], alpha=0.3)
    
    # 2. 損失曲線
    if conv_data['losses']:
        axes[0, 1].plot(conv_data['episodes'], conv_data['losses'], 'r-', linewidth=2, alpha=0.8)
        axes[0, 1].set_title('📉 Loss Convergence', fontweight='bold')
        axes[0, 1].set_xlabel('Episode')
        axes[0, 1].set_ylabel('Average Loss')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].set_yscale('log')
    
    # 3. 勝率推移
    if conv_data['win_rate']:
        axes[0, 2].plot(conv_data['episodes'], conv_data['win_rate'], 'g-', linewidth=2, alpha=0.8)
        axes[0, 2].set_title('🏆 Win Rate Evolution', fontweight='bold')
        axes[0, 2].set_xlabel('Episode')
        axes[0, 2].set_ylabel('Win Rate (%)')
        axes[0, 2].grid(True, alpha=0.3)
        axes[0, 2].set_ylim(0, 100)
    
    # 4. 学習率推移
    if conv_data['learning_rate']:
        axes[0, 3].plot(conv_data['episodes'], conv_data['learning_rate'], 'purple', linewidth=2, alpha=0.8)
        axes[0, 3].set_title('🎛️ Learning Rate Schedule', fontweight='bold')
        axes[0, 3].set_xlabel('Episode')
        axes[0, 3].set_ylabel('Learning Rate')
        axes[0, 3].grid(True, alpha=0.3)
        axes[0, 3].set_yscale('log')
    
    # 5. ゲーム長分布
    axes[1, 0].hist(eval_results['game_lengths'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1, 0].set_title('🎲 Game Length Distribution', fontweight='bold')
    axes[1, 0].set_xlabel('Game Length (moves)')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 6. 報酬分布
    axes[1, 1].hist(eval_results['rewards'], bins=25, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1, 1].set_title('💰 Reward Distribution', fontweight='bold')
    axes[1, 1].set_xlabel('Reward')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    # 7. 性能比較
    train_win = training_results['final_performance']['final_win_rate']
    eval_win = eval_results['win_rate']
    draw_rate = eval_results['draws'] / 100 * 100  # n_games=100想定
    loss_rate = eval_results['losses'] / 100 * 100
    
    performance_data = [train_win, eval_win]
    performance_labels = ['Training', 'Evaluation']
    colors = ['lightgreen', 'lightblue']
    
    bars = axes[1, 2].bar(performance_labels, performance_data, color=colors, alpha=0.8, edgecolor='black')
    axes[1, 2].set_title('🏅 Performance Comparison', fontweight='bold')
    axes[1, 2].set_ylabel('Win Rate (%)')
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].set_ylim(0, 100)
    
    # バーの上に数値表示
    for bar, value in zip(bars, performance_data):
        axes[1, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                        f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # 8. 評価結果サマリー (円グラフ)
    eval_labels = ['Wins', 'Draws', 'Losses']
    eval_sizes = [eval_results['wins'], eval_results['draws'], eval_results['losses']]
    eval_colors = ['lightgreen', 'lightyellow', 'lightcoral']
    
    wedges, texts, autotexts = axes[1, 3].pie(eval_sizes, labels=eval_labels, colors=eval_colors, 
                                             autopct='%1.1f%%', startangle=90)
    axes[1, 3].set_title('🎯 Evaluation Results', fontweight='bold')
    
    # 美しいフォーマット
    for autotext in autotexts:
        autotext.set_color('black')
        autotext.set_fontweight('bold')
    
    plt.tight_layout()
    plt.show()
    
    # 収束分析グラフ
    if conv_data['losses'] and len(conv_data['episodes']) > 1:
        fig2, ax = plt.subplots(1, 1, figsize=(12, 6))
        
        ax2 = ax.twinx()
        
        # 損失（左軸）
        line1 = ax.plot(conv_data['episodes'], conv_data['losses'], 'r-', linewidth=2, alpha=0.8, label='Loss')
        ax.set_xlabel('Episode')
        ax.set_ylabel('Loss', color='red')
        ax.tick_params(axis='y', labelcolor='red')
        ax.set_yscale('log')
        
        # 報酬（右軸）
        line2 = ax2.plot(conv_data['episodes'], conv_data['rewards'], 'b-', linewidth=2, alpha=0.8, label='Reward')
        ax2.set_ylabel('Reward', color='blue')
        ax2.tick_params(axis='y', labelcolor='blue')
        
        # 凡例
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax.legend(lines, labels, loc='center right')
        
        ax.set_title('🔄 Training Convergence Analysis', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()


# =============================================================================
# メイン実行部分
# =============================================================================

print("🎯 完全量子AI学習パイプライン実行開始")
print(f"📅 実行日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 設定確認
if 'config' not in locals():
    print("⚠️ 設定が読み込まれていません。セル2を実行してください。")
    print("⚠️ 一時的にデフォルト設定を使用します。")

    config = {
        "learning_config": {
            "method": "reinforcement",
            "algorithm": "dqn",
            "timestamp": datetime.now().isoformat()
        },
        "module_config": {
            "quantum": {
                "n_qubits": 4,
                "n_layers": 2,
                "embedding_type": "angle",
                "entanglement": "linear",
                "total_params": 24
            },
            "reward": {
                "strategy": "balanced"
            },
            "qmap": {
                "method": "dqn",
                "state_dim": 252,
                "action_dim": 5
            },
            "action_selection": {
                "strategy": "epsilon"
            }
        },
        "hyperparameters": {
            "batch_size": 32,
            "epochs": 500,
            "learning_rate": 0.001,
            "optimizer": "adam"
        }
    }

# パイプライン実行
final_results = complete_quantum_ai_pipeline(
    config=config,
    model_name="quantum_geister_ai_clean_v2",
    max_episodes=600  # VSCode環境では少し短めに
)

print("\n🎉 完全量子AI学習パイプライン完了!")
print(f"🎊 お疲れ様でした！モデルの保存と評価がすべて完了しました。")

🎯 完全量子AI学習パイプライン実行開始
📅 実行日時: 2025-09-23 22:15:07
🚀 完全量子AI学習パイプライン開始
🏗️ Phase 1: モデル初期化
   ✅ モデル作成完了
   📊 総パラメータ数: 4,761
   ⚛️ 量子ビット: 4, レイヤー: 1
   🎮 行動次元: 5

📚 Phase 2: スマート学習実行
🎯 スマート学習開始 (最大600エピソード)


KeyError: 'batch_size'

## 📋 5. 結果の確認と次のステップ

学習完了後、以下のファイルが生成されます：

### 📁 出力ファイル
- **`trained_models/quantum_geister_ai_clean_v2_YYYYMMDD_HHMMSS.pth`**: 学習済みモデル
- **`training_report_YYYYMMDD_HHMMSS.json`**: 包括的な学習レポート

### 🎯 性能指標
- **勝率**: 評価ゲームでの勝利率
- **収束状況**: 損失が収束したかどうか
- **性能グレード**: A (優秀) ~ D (要改善)
- **推奨事項**: 改善提案

### 🚀 次のステップ
1. **モデル改善**: 推奨事項に基づく調整
2. **実戦テスト**: 実際のゲームエンジンでの評価
3. **パラメータ調整**: より良い性能を目指した微調整
4. **WebUI統合**: WebUIシステムとの連携

---

**🎊 VSCode環境での量子AI開発をお楽しみください！**